In [1]:
from pre import process

In [2]:
df = process('train','test')

In [3]:
# make sure that df.shape is 2919,195
df.shape

(2919, 195)

In [4]:
df.head()

,MSSubClass,LotFrontage,LotArea,Street,Utilities,LandSlope,OverallQual,OverallCond,YearBuilt,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,TotalBsmtSF,HeatingQC,CentralAir,GrLivArea,BsmtBath,FullBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,Month,Year,VIX_Close_Mo_Avg,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RM,LotShape_IR1,LotShape_IR2,LotShape_IR3,LandContour_Bnk,LandContour_HLS,LandContour_Low,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Artery,Condition1_Feedr,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Flat,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_ClyTile,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_Stone,Foundation_BrkTil,Foundation_CBlock,Foundation_Slab,Foundation_Stone,Foundation_Wood,Heating_Floor,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,GarageType_2Types,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,Fence_GdPrv,Fence_GdWo,Fence_MnWw,MiscFeature_Gar2,MiscFeature_Othr,MiscFeature_TenC,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Partial,SalePrice
0,60,65.0,8450,1,4.0,3,7,5,2003,196.0,4,3,4,3,1,6,1,856.0,5,1,1710,1.0,2.5,3,1,4,8,8.0,0,0,2,2.0,548.0,3,3,3,0,61,0,0,0,0,0,2,2008,2,2008,25.456000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,208500.0
1,20,80.0,9600,1,4.0,3,6,8,1976,0.0,3,3,4,3,4,5,1,1262.0,5,1,1262,0.5,2.0,3,1,3,6,8.0,1,3,2,2.0,460.0,3,3,3,298,0,0,0,0,0,0,5,2007,5,2007,13.297727,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,181500.0
2,60,68.0,11250

In [ ]:
px.scatter(data_frame=df,x ='GrLivArea',y='SalePrice')

In [5]:
df_out = df[df['GrLivArea']<4000][df['SalePrice']<700000]

In [6]:
from sklearn.linear_model import LinearRegression,Ridge, Lasso,RidgeCV, LassoCV, ElasticNet, HuberRegressor
import numpy as np
ols = LinearRegression()
ridge = Ridge()
lasso = Lasso()
huber = HuberRegressor()
ols.set_params(normalize=True)
ridge.set_params(normalize=True)
lasso.set_params(normalize=True)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000, normalize=True,
      positive=False, precompute=False, random_state=None, selection='cyclic',
      tol=0.0001, warm_start=False)

In [7]:
X_train = df_out.loc[:1459, df_out.columns !='SalePrice']
Y_train = df_out.loc[:1459, df_out.columns =='SalePrice']
X_test = df.loc[1460:, df_out.columns !='SalePrice']

In [8]:
X_test.shape

(1459, 194)

In [9]:
ols.fit(X_train,Y_train)
ols.score(X_train,Y_train)

0.9234280910721279

In [ ]:
Y_train.hist()
np.log(Y_train).hist()

In [10]:
parameters={'alpha':np.linspace(15,20,20)}

from sklearn.model_selection import GridSearchCV
GridSearchCV.set_params(lasso,normalize=True)
grid_lasso=GridSearchCV(lasso, parameters, cv=5)
grid_lasso.fit(X_train, Y_train)

print(grid_lasso.best_params_)

print(grid_lasso.best_score_)


{'alpha': 17.36842105263158}
0.8941155830355872


In [11]:
lasso.set_params(alpha=17.36842105263158,normalize=True)
lasso.fit(X_train, Y_train)
# pd.DataFrame(lasso.coef_).rename(columns={0:'Coefficients'})
# pd.DataFrame(lasso.coef_, columns = df.columns) 
lasso_coefs = pd.DataFrame(lasso.coef_,index=np.array(df_out.iloc[:,:194].columns)).rename(columns={0:'Coefficients'})
pd.DataFrame(lasso.predict(X_test)).to_csv('Submission1.csv')
lasso.score(X_train,Y_train)

0.9136226183377568

In [12]:
dropping = list(lasso_coefs[lasso_coefs['Coefficients']==0].index)
len(dropping)

101

In [13]:
df_out1 = df_out.copy()

In [14]:
df_out1.drop(columns=dropping,inplace=True) #THIS HAPPENED

In [15]:
x_train_out = df_out1.loc[:1459, df_out1.columns !='SalePrice']
y_train_out = df_out1.loc[:1459, df_out1.columns =='SalePrice']
x_test_out = df.loc[1460:, df.columns !='SalePrice'].drop(columns=dropping)

In [16]:
parameters={'alpha':np.linspace(0,10,100)}

from sklearn.model_selection import GridSearchCV
GridSearchCV.set_params(lasso,normalize=True)
grid_lasso=GridSearchCV(lasso, parameters, cv=5)
grid_lasso.fit(x_train_out, y_train_out)

print(grid_lasso.best_params_)

print(grid_lasso.best_score_)

{'alpha': 5.555555555555555}
0.8981794957433793


In [17]:
lasso.set_params(alpha=5.555555555555555,normalize=True)
lasso.fit(df_out1.loc[:1459, df_out1.columns != 'SalePrice'], (df_out1.loc[:1459])['SalePrice'])
# pd.DataFrame(lasso.coef_).rename(columns={0:'Coefficients'})
# pd.DataFrame(lasso.coef_, columns = df.columns) 
lasso2_coefs = pd.DataFrame(lasso.coef_,index=np.array(df_out1.iloc[:,:93].columns)).rename(columns={0:'Coefficients'})
pd.DataFrame(lasso.predict(x_test_out)).to_csv('Submission2.csv')
lasso.score(x_train_out,y_train_out)


0.9191751397109992

In [18]:
ols.fit(X_train,np.log(Y_train))
(np.e**pd.DataFrame(ols.predict(X_test))[[0]]).to_csv('Submission3.csv')
ols.score(X_train,np.log(Y_train)) ##.134, this is without the dropped columns

0.9409898316932449

In [19]:
ols.fit(x_train_out,np.log(y_train_out))
(np.e**pd.DataFrame(ols.predict(x_test_out))[[0]]).to_csv('Submission4.csv')
ols.score(x_train_out,np.log(y_train_out)) ## .12828, with the dropped columns, don't need lasso because alpha is 0

0.9338711416643627

In [64]:
df_out2 = df_out1.copy()

In [65]:
df_out2.shape ### TRANSFORMING X VARIABELES FROM LOGS AND OTHER STUFF

(1456, 94)

In [66]:
x_transform = df_out2.loc[:1459, df_out2.columns !='SalePrice']
y_transform = df_out2.loc[:1459, df_out2.columns =='SalePrice']
x_test_transform = df.loc[1460:, df.columns !='SalePrice'].drop(columns=dropping)

In [67]:
# # Columns to log
df_out2['GrLivArea'] = df_out2['GrLivArea'].apply(lambda x: np.log(x))
df_out2['LotArea'] = df_out2['LotArea'].apply(lambda x: np.log(x))
# Columns to log + 1
df_out2['MasVnrArea'] = df_out2['MasVnrArea'].apply(lambda x: np.log(x+1))
df_out2['OpenPorchSF'] = df_out2['OpenPorchSF'].apply(lambda x: np.log(x+1))
# Columns to root
df_out2['MSSubClass'] = df_out2['MSSubClass'].apply(lambda x: np.sqrt(x))
df_out2['LotFrontage'] = df_out2['LotFrontage'].apply(lambda x: np.sqrt(x))
df_out2['TotalBsmtSF'] = df_out2['TotalBsmtSF'].apply(lambda x: np.sqrt(x))
df_out2['WoodDeckSF'] = df_out2['WoodDeckSF'].apply(lambda x: np.sqrt(x))

In [68]:
df_out2.shape

(1456, 94)

In [69]:
print(x_transform.shape)
print(y_transform.shape)

(1456, 93)
(1456, 1)


In [71]:
ols.fit(x_transform,np.log(y_transform))
# (np.e**pd.DataFrame(ols.predict(x_test_transform))[[0]]).to_csv('Submission5.csv')
ols.score(x_transform,np.log(y_transform))
np.e**ols.predict(x_test_transform)

array([[118403.70812509],
       [155550.51193174],
       [171825.77464495],
       ...,
       [164063.81685663],
       [113471.24001853],
       [220399.55880522]])

In [ ]:
parameters={'alpha':np.linspace(0,10,100)}

from sklearn.model_selection import GridSearchCV
GridSearchCV.set_params(lasso,normalize=True)
grid_lasso=GridSearchCV(lasso, parameters, cv=5)
grid_lasso.fit(x_transform,np.log(y_transform))

print(grid_lasso.best_params_)

print(grid_lasso.best_score_)

In [48]:
lasso.set_params(alpha=0,normalize=True)
lasso.fit(x_transform,np.log(y_transform))
# pd.DataFrame(lasso.coef_).rename(columns={0:'Coefficients'})
# pd.DataFrame(lasso.coef_, columns = df.columns) 
# lasso2_coefs = pd.DataFrame(lasso.coef_,index=np.array(df_out1.iloc[:,:93].columns)).rename(columns={0:'Coefficients'})
# pd.DataFrame(lasso.predict(x_test_out)).to_csv('Submission2.csv')
lasso.score(x_transform,np.log(y_transform))
np.e**lasso.predict(x_test_transform)

array([118403.70812509, 155550.51193174, 171825.77464495, ...,
       164063.81685663, 113471.24001853, 220399.55880522])

In [72]:
parameters={'alpha':np.linspace(0,10,100)}

from sklearn.model_selection import GridSearchCV
GridSearchCV.set_params(ridge,normalize=True)
grid_ridge=GridSearchCV(ridge, parameters, cv=5)
grid_ridge.fit(x_transform,np.log(y_transform))

print(grid_lasso.best_params_)

print(grid_lasso.best_score_)

{'alpha': 5.555555555555555}
0.8981794957433793


/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [81]:
ridge.set_params(alpha=5.5555,normalize=True)
ridge.fit(x_transform,np.log(y_transform))
pd.DataFrame(np.e**grid_ridge.predict(x_test_transform)).to_csv('Submission6.csv')
# np.e**ridge.predict(x_test_transform)